## LSTM을 이용한 네이버 영화 리뷰 분류

### 1. 네이버 영화 리뷰 데이터 이해 및 전처리

In [34]:
#!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [35]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Mecab
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter


#### 1) 데이터 로드

In [36]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="./Data/ratings/ratings_train.txt")  # train 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="./Data/ratings/ratings_test.txt")    # test 데이터 다운로드

('./Data/ratings/ratings_test.txt', <http.client.HTTPMessage at 0x24fce22d960>)

In [37]:
# pandas를 이용해 데이터 로드
train_data = pd.read_table('./Data/ratings/ratings_train.txt')
test_data = pd.read_table('./Data/ratings/ratings_test.txt')

In [38]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력

훈련용 리뷰 개수 : 150000


In [39]:
train_data.head()


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [40]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


#### 2) 데이터 정제하기

In [41]:
# 결측치 확인
train_data.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [42]:
train_data = train_data.dropna(axis=0) # Null 값이 존재하는 행 제거

In [43]:
# document 열과 label 열의 중복을 제외한 값의 개수
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [44]:
# document 열의 중복 제거
train_data = train_data.drop_duplicates(subset=['document'])

In [45]:
train_data.shape

(146182, 3)

In [46]:
print((train_data.label == 0).sum())
print((train_data.label == 1).sum())

73342
72840


In [47]:
# 한글과 공백을 제외하고 모두 제거
# []: 문자 클래스
# ^: not
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [48]:
# ^: 문자열의 시작
# +: 1개 이상 연속된 문자
train_data['document'] = train_data['document'].str.replace('^ +', "", regex=True) # white space 데이터를 empty value로 변경
train_data = train_data.replace({'document': ''}, np.nan)
print(train_data.isnull().sum())

id            0
document    789
label         0
dtype: int64


In [49]:
train_data = train_data.dropna(axis=0)

In [50]:
## test 데이터 전처리
test_data.drop_duplicates(subset = ['document']) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 한글만 남기기
test_data['document'] = test_data['document'].str.replace('^ +', "", regex=True) # 공백은 empty 값으로 변경
test_data = test_data.replace({'document': ''}, np.nan) # 공백은 Null 값으로 변경
test_data = test_data.dropna(axis=0) # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 49575
